## Preprocessing

In [76]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [77]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])

In [78]:
# Determine the number of unique values in each column.
unique_values = application_df.nunique()
print(unique_values)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [79]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_counts = application_df['APPLICATION_TYPE'].value_counts()
print(application_counts)

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64


In [80]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
threshold = 50
application_types_to_replace = application_counts[application_counts < threshold].index

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

,count
APPLICATION_TYPE,
T3,27037
T4,1542
T6,1216
T5,1173
T19,1065
T8,737
T7,725
T10,528
T9,156


In [81]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_counts = application_df['CLASSIFICATION'].value_counts()
print(classification_counts)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64


In [82]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts_gt_1 = classification_counts[classification_counts > 1]

print(classification_counts_gt_1)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64


In [83]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
threshold = 105
classifications_to_replace = classification_counts[classification_counts < threshold].index

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

,count
CLASSIFICATION,
C1000,17326
C2000,6074
C1200,4837
C3000,1918
C2100,1883
C7000,777
Other,773
C1700,287
C4000,194


In [84]:
# Convert categorical data to numeric with `pd.get_dummies`
categorical_cols = ['APPLICATION_TYPE', 'INCOME_AMT', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'SPECIAL_CONSIDERATIONS']
application_df_dummies = pd.get_dummies(application_df, columns=categorical_cols, drop_first=True, dtype='int')

In [85]:
application_df_dummies.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T13,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,CLASSIFICATION_C7000,CLASSIFICATION_Other,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,1,108590,1,0,0,0,1,0,0,0,...,0,0,0,0,1,0,1,0,0,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [86]:
application_df_dummies.columns

Index(['STATUS', 'ASK_AMT', 'IS_SUCCESSFUL', 'APPLICATION_TYPE_T10',
       'APPLICATION_TYPE_T13', 'APPLICATION_TYPE_T19', 'APPLICATION_TYPE_T3',
       'APPLICATION_TYPE_T4', 'APPLICATION_TYPE_T5', 'APPLICATION_TYPE_T6',
       'APPLICATION_TYPE_T7', 'APPLICATION_TYPE_T8', 'APPLICATION_TYPE_T9',
       'INCOME_AMT_1-9999', 'INCOME_AMT_10000-24999',
       'INCOME_AMT_100000-499999', 'INCOME_AMT_10M-50M', 'INCOME_AMT_1M-5M',
       'INCOME_AMT_25000-99999', 'INCOME_AMT_50M+', 'INCOME_AMT_5M-10M',
       'AFFILIATION_Family/Parent', 'AFFILIATION_Independent',
       'AFFILIATION_National', 'AFFILIATION_Other', 'AFFILIATION_Regional',
       'CLASSIFICATION_C1200', 'CLASSIFICATION_C1270', 'CLASSIFICATION_C1700',
       'CLASSIFICATION_C2000', 'CLASSIFICATION_C2100', 'CLASSIFICATION_C3000',
       'CLASSIFICATION_C4000', 'CLASSIFICATION_C5000', 'CLASSIFICATION_C7000',
       'CLASSIFICATION_Other', 'USE_CASE_Heathcare', 'USE_CASE_Other',
       'USE_CASE_Preservation', 'USE_CASE_ProductD

In [87]:
# Split our preprocessed data into our features and target arrays
X = application_df_dummies.drop(columns=['IS_SUCCESSFUL'])
y = application_df_dummies['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78, test_size=0.2)

In [88]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [89]:
## OPTIMIZATION ATTEMPT 1
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

# Define the number of input features
number_input_features = X_train.shape[1]  # Number of features in the training data

# Define the number of hidden nodes for each layer
hidden_nodes_layer1 = 10  # Number of nodes in the first hidden layer
hidden_nodes_layer2 = 7   # Number of nodes in the second hidden layer

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, activation='relu', input_dim=number_input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_35 (Dense)                     │ (None, 10)                  │             440 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_36 (Dense)                     │ (None, 7)                   │              77 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_37 (Dense)                     │ (None, 1)                   │               8 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 525 (2.05 KB)

 Trainable params: 525 (2.05 KB)

 Non-trainable params: 0 (0.00 B)

In [90]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [92]:
# Create a callback to save the model's weights every 5 epochs
checkpoint_callback = ModelCheckpoint(
    filepath='model.weights.h5',  # Filepath to save the model weights
    save_weights_only=True,         # Save only the weights
    save_freq='epoch',              # Save at the end of every epoch
    verbose=1                      # Verbosity mode, 1 = progress messages
)

# Train the model
fit_model = nn.fit(X_train, y_train, epochs=200, shuffle=True, verbose=2, callbacks=[checkpoint_callback])



Epoch 1/200

Epoch 1: saving model to model.weights.h5
858/858 - 4s - 5ms/step - accuracy: 0.4985 - loss: 4689.4014
Epoch 2/200

Epoch 2: saving model to model.weights.h5
858/858 - 1s - 1ms/step - accuracy: 0.5085 - loss: 5400.9023
Epoch 3/200

Epoch 3: saving model to model.weights.h5
858/858 - 1s - 1ms/step - accuracy: 0.5181 - loss: 2794.5659
Epoch 4/200

Epoch 4: saving model to model.weights.h5
858/858 - 1s - 1ms/step - accuracy: 0.5157 - loss: 4068.2375
Epoch 5/200

Epoch 5: saving model to model.weights.h5
858/858 - 2s - 2ms/step - accuracy: 0.5241 - loss: 4301.3882
Epoch 6/200

Epoch 6: saving model to model.weights.h5
858/858 - 2s - 2ms/step - accuracy: 0.5068 - loss: 5671.5566
Epoch 7/200

Epoch 7: saving model to model.weights.h5
858/858 - 1s - 2ms/step - accuracy: 0.5102 - loss: 4716.2334
Epoch 8/200

Epoch 8: saving model to model.weights.h5
858/858 - 2s - 2ms/step - accuracy: 0.5264 - loss: 3837.2300
Epoch 9/200

Epoch 9: saving model to model.weights.h5
858/858 - 2s - 2m

In [93]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - 2ms/step - accuracy: 0.3181 - loss: 1.5839
Loss: 1.5839298963546753, Accuracy: 0.3180758059024811


In [94]:
## OPTIMIZATION ATTEMPT 2: Added a third hidden layer

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

# Define the number of input features
number_input_features = X_train.shape[1]  # Number of features in the training data

# Define the number of hidden nodes for each layer
hidden_nodes_layer1 = 10  # Number of nodes in the first hidden layer
hidden_nodes_layer2 = 7   # Number of nodes in the second hidden layer
hidden_nodes_layer3 = 7

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, activation='relu', input_dim=number_input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_38 (Dense)                     │ (None, 10)                  │             440 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_39 (Dense)                     │ (None, 7)                   │              77 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_40 (Dense)                     │ (None, 7)                   │              56 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_41 (Dense)                     │ (None, 1)                   │               8 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 581 (2.27 KB)

 Trainable params: 581 (2.27 KB)

 Non-trainable params: 0 (0.00 B)

In [95]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [96]:
# Train the model
fit_model = nn.fit(X_train, y_train, epochs=50, shuffle=True, verbose=2)

Epoch 1/50
858/858 - 3s - 4ms/step - accuracy: 0.4972 - loss: 10909.4043
Epoch 2/50
858/858 - 1s - 1ms/step - accuracy: 0.5060 - loss: 1635.1864
Epoch 3/50
858/858 - 1s - 1ms/step - accuracy: 0.5061 - loss: 810.0239
Epoch 4/50
858/858 - 1s - 1ms/step - accuracy: 0.4991 - loss: 522.3023
Epoch 5/50
858/858 - 1s - 1ms/step - accuracy: 0.5060 - loss: 351.7372
Epoch 6/50
858/858 - 1s - 1ms/step - accuracy: 0.5149 - loss: 124.1048
Epoch 7/50
858/858 - 1s - 1ms/step - accuracy: 0.5131 - loss: 676.7781
Epoch 8/50
858/858 - 1s - 1ms/step - accuracy: 0.5259 - loss: 130.2225
Epoch 9/50
858/858 - 1s - 2ms/step - accuracy: 0.5268 - loss: 10.1193
Epoch 10/50
858/858 - 2s - 2ms/step - accuracy: 0.5323 - loss: 0.6911
Epoch 11/50
858/858 - 2s - 2ms/step - accuracy: 0.5323 - loss: 0.6911
Epoch 12/50
858/858 - 1s - 2ms/step - accuracy: 0.5323 - loss: 0.6911
Epoch 13/50
858/858 - 1s - 1ms/step - accuracy: 0.5323 - loss: 0.6911
Epoch 14/50
858/858 - 1s - 1ms/step - accuracy: 0.5323 - loss: 0.6911
Epoch 15/

In [97]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - 2ms/step - accuracy: 0.5329 - loss: 0.6706
Loss: 0.6705887913703918, Accuracy: 0.5329446196556091


In [98]:
## OPTIMIZATION ATTEMPT 3: Reduced the number of columns, added a fourth hidden layer, and reduced the number of bins

# Split our preprocessed data into our features and target arrays
X = application_df_dummies.drop(columns=['IS_SUCCESSFUL', 'ASK_AMT', 'SPECIAL_CONSIDERATIONS_Y'])
y = application_df_dummies['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78, test_size=0.2)

In [99]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [100]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

# Define the number of input features
number_input_features = X_train.shape[1]  # Number of features in the training data

# Define the number of hidden nodes for each layer
hidden_nodes_layer1 = 20  # Number of nodes in the first hidden layer
hidden_nodes_layer2 = 15   # Number of nodes in the second hidden layer
hidden_nodes_layer3 = 15  # Number of  nodes in the third hidden layer
hidden_nodes_layer4 = 20 # Number of nodes in the fourth hidden layer
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, activation='relu', input_dim=number_input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation='relu'))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_42 (Dense)                     │ (None, 20)                  │             840 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_43 (Dense)                     │ (None, 15)                  │             315 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_44 (Dense)                     │ (None, 15)                  │             240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_45 (Dense)                     │ (None, 20)                  │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_46 (Dense)                     │ (None, 1)                   │              21 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,736 (6.78 KB)

 Trainable params: 1,736 (6.78 KB)

 Non-trainable params: 0 (0.00 B)

In [101]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [102]:
# Train the model
fit_model = nn.fit(X_train, y_train, epochs=30, shuffle=True, verbose=2)

Epoch 1/30
858/858 - 3s - 4ms/step - accuracy: 0.7103 - loss: 0.5833
Epoch 2/30
858/858 - 2s - 2ms/step - accuracy: 0.7282 - loss: 0.5565
Epoch 3/30
858/858 - 2s - 3ms/step - accuracy: 0.7307 - loss: 0.5529
Epoch 4/30
858/858 - 2s - 2ms/step - accuracy: 0.7325 - loss: 0.5502
Epoch 5/30
858/858 - 1s - 1ms/step - accuracy: 0.7322 - loss: 0.5483
Epoch 6/30
858/858 - 1s - 1ms/step - accuracy: 0.7339 - loss: 0.5475
Epoch 7/30
858/858 - 1s - 1ms/step - accuracy: 0.7326 - loss: 0.5463
Epoch 8/30
858/858 - 1s - 1ms/step - accuracy: 0.7328 - loss: 0.5459
Epoch 9/30
858/858 - 1s - 1ms/step - accuracy: 0.7329 - loss: 0.5453
Epoch 10/30
858/858 - 1s - 1ms/step - accuracy: 0.7338 - loss: 0.5456
Epoch 11/30
858/858 - 1s - 2ms/step - accuracy: 0.7337 - loss: 0.5441
Epoch 12/30
858/858 - 2s - 2ms/step - accuracy: 0.7350 - loss: 0.5436
Epoch 13/30
858/858 - 2s - 2ms/step - accuracy: 0.7359 - loss: 0.5432
Epoch 14/30
858/858 - 1s - 1ms/step - accuracy: 0.7356 - loss: 0.5427
Epoch 15/30
858/858 - 1s - 1m

In [103]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - 2ms/step - accuracy: 0.5102 - loss: 2.7969
Loss: 2.7968530654907227, Accuracy: 0.5102040767669678


In [104]:
## OPTIMIZATION ATTEMPT 4: Used different activation functions for the hidden layers

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

# Define the number of input features
number_input_features = X_train.shape[1]  # Number of features in the training data

# Define the number of hidden nodes for each layer
hidden_nodes_layer1 = 20  # Number of nodes in the first hidden layer
hidden_nodes_layer2 = 15   # Number of nodes in the second hidden layer
hidden_nodes_layer3 = 15  # Number of  nodes in the third hidden layer
hidden_nodes_layer4 = 20 # Number of nodes in the fourth hidden layer
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, activation='tanh', input_dim=number_input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='tanh'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation='tanh'))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation='tanh'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_47 (Dense)                     │ (None, 20)                  │             840 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_48 (Dense)                     │ (None, 15)                  │             315 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_49 (Dense)                     │ (None, 15)                  │             240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_50 (Dense)                     │ (None, 20)                  │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_51 (Dense)                     │ (None, 1)                   │              21 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,736 (6.78 KB)

 Trainable params: 1,736 (6.78 KB)

 Non-trainable params: 0 (0.00 B)

In [105]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [106]:
# Train the model
fit_model = nn.fit(X_train, y_train, epochs=40, shuffle=True, verbose=2)

Epoch 1/40
858/858 - 3s - 3ms/step - accuracy: 0.7200 - loss: 0.5799
Epoch 2/40
858/858 - 1s - 1ms/step - accuracy: 0.7291 - loss: 0.5621
Epoch 3/40
858/858 - 1s - 1ms/step - accuracy: 0.7302 - loss: 0.5572
Epoch 4/40
858/858 - 2s - 2ms/step - accuracy: 0.7306 - loss: 0.5544
Epoch 5/40
858/858 - 2s - 2ms/step - accuracy: 0.7299 - loss: 0.5531
Epoch 6/40
858/858 - 2s - 2ms/step - accuracy: 0.7310 - loss: 0.5525
Epoch 7/40
858/858 - 1s - 1ms/step - accuracy: 0.7315 - loss: 0.5514
Epoch 8/40
858/858 - 1s - 1ms/step - accuracy: 0.7319 - loss: 0.5505
Epoch 9/40
858/858 - 1s - 1ms/step - accuracy: 0.7321 - loss: 0.5500
Epoch 10/40
858/858 - 1s - 1ms/step - accuracy: 0.7315 - loss: 0.5500
Epoch 11/40
858/858 - 1s - 1ms/step - accuracy: 0.7331 - loss: 0.5496
Epoch 12/40
858/858 - 1s - 1ms/step - accuracy: 0.7322 - loss: 0.5496
Epoch 13/40
858/858 - 1s - 2ms/step - accuracy: 0.7322 - loss: 0.5495
Epoch 14/40
858/858 - 3s - 3ms/step - accuracy: 0.7311 - loss: 0.5488
Epoch 15/40
858/858 - 1s - 1m

In [107]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - 2ms/step - accuracy: 0.6372 - loss: 0.6664
Loss: 0.6663787961006165, Accuracy: 0.6371719837188721


In [108]:
## OPTIMIZATION ATTEMPT 5: Used different activation functions for the hidden layers and increased the number of bins for rare occurences in columns

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

# Define the number of input features
number_input_features = X_train.shape[1]  # Number of features in the training data

# Define the number of hidden nodes for each layer
hidden_nodes_layer1 = 20  # Number of nodes in the first hidden layer
hidden_nodes_layer2 = 15   # Number of nodes in the second hidden layer
hidden_nodes_layer3 = 15  # Number of  nodes in the third hidden layer
hidden_nodes_layer4 = 20 # Number of nodes in the fourth hidden layer
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, activation='tanh', input_dim=number_input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='tanh'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation='tanh'))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation='tanh'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_52 (Dense)                     │ (None, 20)                  │             840 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_53 (Dense)                     │ (None, 15)                  │             315 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_54 (Dense)                     │ (None, 15)                  │             240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_55 (Dense)                     │ (None, 20)                  │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_56 (Dense)                     │ (None, 1)                   │              21 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,736 (6.78 KB)

 Trainable params: 1,736 (6.78 KB)

 Non-trainable params: 0 (0.00 B)

In [109]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [110]:
# Train the model
fit_model = nn.fit(X_train, y_train, epochs=40, shuffle=True, verbose=2)

Epoch 1/40
858/858 - 3s - 3ms/step - accuracy: 0.7122 - loss: 0.5845
Epoch 2/40
858/858 - 1s - 1ms/step - accuracy: 0.7251 - loss: 0.5668
Epoch 3/40
858/858 - 1s - 1ms/step - accuracy: 0.7265 - loss: 0.5609
Epoch 4/40
858/858 - 1s - 1ms/step - accuracy: 0.7285 - loss: 0.5566
Epoch 5/40
858/858 - 1s - 1ms/step - accuracy: 0.7278 - loss: 0.5549
Epoch 6/40
858/858 - 2s - 2ms/step - accuracy: 0.7290 - loss: 0.5535
Epoch 7/40
858/858 - 2s - 3ms/step - accuracy: 0.7296 - loss: 0.5522
Epoch 8/40
858/858 - 2s - 3ms/step - accuracy: 0.7297 - loss: 0.5519
Epoch 9/40
858/858 - 1s - 1ms/step - accuracy: 0.7301 - loss: 0.5505
Epoch 10/40
858/858 - 1s - 1ms/step - accuracy: 0.7305 - loss: 0.5500
Epoch 11/40
858/858 - 1s - 1ms/step - accuracy: 0.7316 - loss: 0.5497
Epoch 12/40
858/858 - 1s - 1ms/step - accuracy: 0.7316 - loss: 0.5498
Epoch 13/40
858/858 - 1s - 1ms/step - accuracy: 0.7322 - loss: 0.5492
Epoch 14/40
858/858 - 1s - 2ms/step - accuracy: 0.7321 - loss: 0.5482
Epoch 15/40
858/858 - 2s - 2m

In [111]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - 2ms/step - accuracy: 0.6630 - loss: 0.6429
Loss: 0.642932653427124, Accuracy: 0.6629737615585327


In [112]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity_Optimization.h5')